# Import libraries

In [1]:
import os, glob, csv, sys, re
from os import path
from urllib.parse import urlparse
import pandas as pd
from bs4 import BeautifulSoup
from tld import get_tld

# Define directories

In [2]:
number = '11'
userhome = os.path.expanduser('~')
dataset = userhome + r'/mining-forum/data/forumfiles_' + number + '/'
analyze_dir = userhome + r'/mining-forum/analyze/'

# Open clean dataset

In [3]:
data_file = pd.read_csv(analyze_dir + 'csv_files/clean_files/clean_files_forum_' + number + '.csv')
data_file

file_number       message_id
0         1000002  message #305932
1         1000019  message #305935
2         1000021  message #305937
3         1000033  message #305941
4         1000043  message #305943
5         1000058  message #305944
6         1000060  message #305945
7         1000072  message #305946
8         1000088  message #305949
9         1000094  message #305956
10        1000108  message #305959
11        1000113  message #305962
12        1000122  message #305967
13        1000130  message #305969
14        1000149  message #305971
15        1000163  message #305992
16        1000171  message #305994
17        1000185  message #305996
18        1000199  message #306001
19        1000208  message #306007
20        1000213  message #306008
21        1000222  message #306010
22        1000230  message #306011
23        1000249  message #306013
24        1000252  message #306014
25        1000263  message #306016
26        1000271  message #306017
27        1000285  message #306018
28        1000299  message #306019
29        1000302  message #306020
...           ...              ...
9494      1096299  message #334961
9495      1096305  message #334962
9496      1096319  message #334963
9497      1096326  message #334971
9498      1096334  message #334974
9499      1096344  message #334979
9500      1096358  message #334982
9501      1096367  message #334990
9502      1096375  message #334992
9503      1096388  message #334996
9504      1096393  message #334999
9505      1096403  message #335001
9506      1096418  message #335007
9507      1096420  message #335010
9508      1096432  message #335012
9509      1096442  message #335014
9510      1096459  message #335015
9511      1096461  message #335022
9512      1096473  message #335027
9513      1096489  message #335031
9514      1096495  message #335033
9515      1096509  message #335034
9516      1096512  message #335035
9517      1096523  message #335038
9518      1096531  message #335040
9519      1096548  message #335041
9520      1096553  message #335044
9521      1096562  message #335054
9522      1096570  message #335055
9523      1096582  message #335056

[9524 rows x 2 columns]

In [4]:
data_file.iloc[1][0]

1000019

# Get links from files

In [5]:
url_data = []
for file_num in range(0,len(data_file)):
    sys.stdout.write('\rExtracting file number: %i' %(file_num+1) + ' / %i' %(len(data_file)))
    sys.stdout.flush()
    file = dataset + str(data_file.iloc[file_num][0])
    f = open(file)
    f = f.read()
    
    try:
        soup = BeautifulSoup(f, 'html.parser')
        topic = soup.find('a', attrs={'class':'MsgSubText'})
        for title in topic.stripped_strings:
            title = title
        
        msid = []
        soup = BeautifulSoup(f, 'html.parser')
        for message_id in soup.find_all('td', attrs={'class':'MsgR1 vt al MsgSubText'}):
            for mid in message_id.find_all('span', attrs={'class':'SmallText'}):
                for msgid in mid.find_all('a'):
                    msid.append(msgid.text)
        msid = msid[0]

        data = [data_file.iloc[file_num][0], msid, title]
        soup = BeautifulSoup(f, 'html.parser')
        for msg in soup.find_all('span', attrs={'class':'MsgBodyText'}):
            for atag in msg.find_all('a'):
                ss = atag.text.replace(' ','')
                parser = []
                try:
                    link = re.search('^https?:\/\/.*[\r\n]*[^.\\,:;]', ss)
                    link = link.group()
                    domain = urlparse(link)
                    tld_dt = get_tld(link)
                    parser = [link, domain.netloc, tld_dt]
                    data.insert(len(data),parser)
                except:
                    pass
        n = 3
        link_name = []
        for i in range (0,len(data)-3):
            if data[n][0] not in link_name:
                url_data.append([data[0], data[1], data[2], data[n][0], data[n][1], data[n][2]])
                link_name.append(data[n][0])
            else:
                pass
            n = n + 1
        
    except:
        pass

print ('\nFiles extraction are finished')

Extracting file number: 9524 / 9524
Files extraction are finished


In [6]:
url_data

[[1000002,
  'message #305932',
  'Simple Text editor in RCP',
  'http://www.eclipse.org/eclipse/platform-text/development/rcp/examples/index.html',
  'www.eclipse.org',
  'org'],
 [1000058,
  'message #305944',
  'Develop multiple versions of a plugin at once',
  'http://www.restlet.org',
  'www.restlet.org',
  'org'],
 [1000058,
  'message #305944',
  'Develop multiple versions of a plugin at once',
  'https://bugs.eclipse.org/bugs/show_bug.cgi?id=151044',
  'bugs.eclipse.org',
  'org'],
 [1000199,
  'message #306001',
  'My External Tools command is no longer saving dirty editors in Eclipse 3.2',
  'https://bugs.eclipse.org/bugs/show_bug.cgi?id=146298',
  'bugs.eclipse.org',
  'org'],
 [1000208,
  'message #306007',
  'PDE extension schema editor help?',
  'https://bugs.eclipse.org/bugs/show_bug.cgi?id=151025',
  'bugs.eclipse.org',
  'org'],
 [1000360,
  'message #306042',
  'ProgressManager throwing strange Job exception',
  'https://bugs.eclipse.org/bugs/show_bug.cgi?id=123800',


In [7]:
with open (analyze_dir + 'csv_files/links_list/link_list_forumfiles_' + number + '.csv', 'w') as csvfile:
    writers = csv.writer(csvfile, delimiter = ",")
    header = ['file_number', 'message_id', 'topic', 'link', 'domain', 'top_level_domain']
    writers.writerow(header)
    for item in url_data:
        writers.writerow(item)
print ('CSV file has been created')

CSV file has been created


In [8]:
link_data = pd.read_csv(analyze_dir + 'csv_files/links_list/link_list_forumfiles_' + number + '.csv')
link_data

file_number       message_id  \
0         1000002  message #305932   
1         1000058  message #305944   
2         1000058  message #305944   
3         1000199  message #306001   
4         1000208  message #306007   
5         1000360  message #306042   
6         1000360  message #306042   
7         1000524  message #306076   
8         1000565  message #306089   
9         1000609  message #306096   
10        1000609  message #306096   
11        1000665  message #306107   
12        1000683  message #306112   
13        1000683  message #306112   
14        1000718  message #306117   
15        1000759  message #306121   
16        1000774  message #306126   
17        1000822  message #306134   
18        1000822  message #306134   
19        1000878  message #306146   
20        1000902  message #306150   
21        1000902  message #306150   
22        1000972  message #306169   
23        1001016  message #306178   
24        1001035  message #306185   
25        1001074  message #306195   
26        1001074  message #306195   
27        1001080  message #306196   
28        1001092  message #306197   
29        1001092  message #306197   
...           ...              ...   
7468      1096367  message #334990   
7469      1096375  message #334992   
7470      1096418  message #335007   
7471      1096418  message #335007   
7472      1096418  message #335007   
7473      1096418  message #335007   
7474      1096418  message #335007   
7475      1096418  message #335007   
7476      1096418  message #335007   
7477      1096432  message #335012   
7478      1096432  message #335012   
7479      1096432  message #335012   
7480      1096432  message #335012   
7481      1096432  message #335012   
7482      1096432  message #335012   
7483      1096473  message #335027   
7484      1096473  message #335027   
7485      1096473  message #335027   
7486      1096473  message #335027   
7487      1096473  message #335027   
7488      1096473  message #335027   
7489      1096473  message #335027   
7490      1096473  message #335027   
7491      1096523  message #335038   
7492      1096523  message #335038   
7493      1096523  message #335038   
7494      1096531  message #335040   
7495      1096548  message #335041   
7496      1096548  message #335041   
7497      1096570  message #335055   

                                                  topic  \
0                             Simple Text editor in RCP   
1         Develop multiple versions of a plugin at once   
2         Develop multiple versions of a plugin at once   
3     My External Tools command is no longer saving ...   
4                     PDE extension schema editor help?   
5        ProgressManager throwing strange Job exception   
6        ProgressManager throwing strange Job exception   
7                           Using FileDialog for SaveAs   
8                variables view & debugging jdk classes   
9              string translation for dynamic variables   
10             string translation for dynamic variables   
11                [HowTo] add F1-help to toolbar items?   
12                     Workspace compatibility Question   
13                     Workspace compatibility Question   
14    react to "refresh" in project explorer and ref...   
15                      help with releng.eclipsebuilder   
16                     How to create a performance test   
17    [Targets]: No targets could be determined for ...   
18    [Targets]: No targets could be determined for ...   
19    Launching framework depends on JDT in Eclipse 3.2   
20                  How to force update of Decorations?   
21                  How to force update of Decorations?   
22                                       Fragment files   
23    Feature Request 151629 - Add com.jcraft.jsch t...   
24    Does eclipse provide any framework for launchi...   
25                Serving external files as online help   
26                Serving external files as online help 